# Introduction to Quantum Safe Cryptography

This notebook serves as a compilation of the hands-on examples provided in the 'Introduction to Quantum Safe Cryptography' provided by IBM.

Requirements:

-   'cryptography' library 

## CHFs: Cryptographic HASH FUNCTIONS

### The SHA-256 algorithm

Let us demonstrate cryptographic hashing my making use of the well-known SHA-256 algorithm, widely used in modern technologies such as the Blockchain (cryptocurrencies), digital signatures and certificates.

A good hash function will produce digests with large differences even for input strings with small variations.

In [1]:
# Begin by importing the necessary modules

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

We generate now two very similar messages which we will feed to the hash function to compare the outputs. 

In [2]:
#Helper function that returns the number of characters different in two strings

def char_diff(str1, str2):
    return sum ( str1[i] != str2[i] for i in range(len(str1)) )

# Messages to be hashed

message_1 = b"Buy 10000 shares of WXYZ stock now!"
message_2 = b"Buy 10000 shares of VXYZ stock now!"

print(f"The two messages differ by { char_diff(message_1, message_2)} characters")

The two messages differ by 1 characters


We now have to instantiate a hash object and update it with the message to be hashed. We then finalize the hash and print the digest. This involves the calls to three methods.

In [3]:
# Create new SHA-256 hash objects, one for each message

chf_1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
chf_2 = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Update each hash object with the bytes of the corresponding message

chf_1.update(message_1)
chf_2.update(message_2)

# Finalize the hash process and obtain the digests

digest_1 = chf_1.finalize()
digest_2 = chf_2.finalize()

In [4]:
digest_1

b'n\x0eba\xb7\x13\x1b\xd8\x0f\xfd\xb2\xa4\xd4/\x9d\x04&65\x0eE\xe1\x84\xb9/\xcb\xcc\x96F\xea\xf1\xe7'

We need to convert the digests to some more readable (or printeable) format, e.g. hexadecimal strings.

In [5]:
#Convert the resulting hash to hexadecimal strings for convenient printing

digest_1_str = digest_1.hex()
digest_2_str = digest_2.hex()

#Print out the digests as strings 

print(f"digest-1: {digest_1_str}")
print(f"digest-2: {digest_2_str}")

print(f"The two digests differ by { char_diff(digest_1_str, digest_2_str)} characters")

digest-1: 6e0e6261b7131bd80ffdb2a4d42f9d042636350e45e184b92fcbcc9646eaf1e7
digest-2: 6b0abb368c3a1730f935b68105e3f3ae7fd43d7e786d3ed3503dbb45c74ada46
The two digests differ by 57 characters


This shows the 'avalanche effect' in the SHA-256 algorithm. We started with two strings difering only in one character. The digests then differ by 57 characters!

## SYMMETRIC KEY CRYPTOGRAPHY (SKC)

### Illustration of Symmetric Key Encryption (SKE)

We illustrate the implementation of the encrypt and decrypt operations using the classical cipher Caesar and the more modern Advanced Encryption System  (AES). The latter has been the standard for symmetric key encryption since 2001.

In [6]:
# import the required crypto functions which will be demonstrated later
from secretpy import Caesar

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from functools import reduce
import numpy as np

In [7]:
# We now write down a message for encryption

plaintext=u"nobody can see this message. in particular, not my wife."

print(f"\nGiven plaintext: {plaintext}")


Given plaintext: nobody can see this message. in particular, not my wife.


#### Caesar shift cipher:

Caesar shift encryption involves defining

 - An alphabet of possible characters to encode
 - A shift value which can be between 0 (unencrypted) and the length of the alphabet. We consider this the key.

It is known as a monoalphabetic substitution cipher since each letter of the plain text is substituted with another in the ciphertext.

In [8]:
# initialize the required python object for doing Caesar shift encryption
caesar_cipher = Caesar()

# Define the shift, ie the key
caesar_key = 5 
print(f"Caesar shift secret key: {caesar_key}")

# Define the alphabet. Note that symbols, special characters and Capital letters must be included in the alphabet.

alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',')
print(f"alphabet: {alphabet}")

Caesar shift secret key: 5
alphabet: ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',')


Let's encrypt and check the ciphertext

In [9]:
caeser_ciphertext = caesar_cipher.encrypt(plaintext, caesar_key, alphabet)
print(f"Encrypted caeser shift ciphertext: {caeser_ciphertext}")

Encrypted caeser shift ciphertext: stgtiachfscxjjcymnxcrjxxfljdcnscufwynhzqfwecstycrac.nkjd


Nice! Now, we can decrypt the message. We have to use the same key.

In [10]:
caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: nobody can see this message. in particular, not my wife.



Decryption fails if using a different key (shift value)

In [11]:
caesar_key_2 = 10

caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key_2, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: ijzj.tv yivn,,vocdnvh,nnyb,wvdivkymod pgymxvijovhtvrda,w



#### Advanced encryption standard (AES) cipher

We now encrypt the plain text using AES, a popular symmetric key encryption algorithm.

We start by creating the key, in this case, a random 16-letter string.

In [12]:
# lamba defines an inline function in this case that takes two values a,b with the resulting expression of a+b
# reduce uses a two-argument function(above), and applies this to all the entries in the list (random alphabet characters) cumulatively

alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')

aes_key = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])

print(f'AES secret key: {aes_key}')

AES secret key: r yeoqkydifpmobd


AES supports multiple operating modes and requires we specify which to use.

We choose the Cipher Block Chaining (CBC) mode provided by the
class of the library. The CBC mode of AES uses randomness for additional security. This requires specifying a random Initialization Vector (IV), also called a . We will use a random string for this as well, just like we did for the key.

In [13]:
aes_initialization_vector = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])
print(f"AES initialization vector: {aes_initialization_vector}")

AES initialization vector: gkyp xbhg xhdcoe


We can now instantiate an AES cipher on behalf of the sender of the secret message.  

We will then encrypt the plain text to send.

In [14]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes

sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

plaintext=u"nobody can see this message now or never"

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process

aes_ciphertext = aes_encryptor.update(bytes(plaintext, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes

print(f"Encrypted AES ciphertext: {aes_ciphertext}")

ValueError: The length of the provided data is not a multiple of the block length.

Oh, an error. Let's check the length of the plaintext.

In [15]:
# Convierte el texto plano a bytes
plaintext_bytes = bytes(plaintext, 'utf-8')

print(f"\nGiven plaintext: {plaintext}")

# Imprime la longitud del texto plano en bytes
print(f"Plaintext length: {len(plaintext_bytes)} bytes")

# Comprueba si la longitud es múltiplo de 16
if len(plaintext_bytes) % 16 == 0:
    print("Plaintext length is a multiple of the block lenght (16 bytes).")
else:
    print("Plaintext length is a multiple of the block lenght (16 bytes).")



Given plaintext: nobody can see this message now or never
Plaintext length: 40 bytes
Plaintext length is a multiple of the block lenght (16 bytes).


In [16]:
# Contar el número de caracteres ASCII
ascii_count = sum(1 for char in plaintext if ord(char) < 128)

# Imprimir el número de caracteres ASCII
print(f"Número de caracteres ASCII: {ascii_count}")

Número de caracteres ASCII: 40


We can simply generate a more suitable message, with number of ASCII characters (or byte length) proportional to 16. We can also circumvent this issue by automatic padding of the original message.

In [17]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes

sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# New message with proper number of ASCII characters

plaintext_=u"no one except my dude can never see this message"

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process

aes_ciphertext_ = aes_encryptor.update(bytes(plaintext_, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes

print(f"Encrypted AES ciphertext: {aes_ciphertext_}")

Encrypted AES ciphertext: b"\x1aV\x84\x1a\xb1\x15Z\xa4\x8d4\xf4Y\xa1\xe6'\xd6\x9c)#kW\xc1\x01\xe8e\xc6\xc2\x0e\xff\xbdQ\x8e\x82{g\x9c\xa6\xa2\xb4`\xbf\x84$h\xb7\xac\xe0\xbf"


In [18]:
# We implement padding to make the message length a multiple of 16 bytes

from cryptography.hazmat.primitives import padding

sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# Tamaño del bloque de AES es 128 bits (16 bytes)
block_size = 128

# Crea un objeto padder para agregar el relleno PKCS7
padder = padding.PKCS7(block_size).padder()

# Aplica el padding al texto plano
padded_plaintext = padder.update(bytes(plaintext, 'utf-8')) + padder.finalize()

# Encripta el texto con padding
aes_ciphertext = aes_encryptor.update(padded_plaintext) + aes_encryptor.finalize()

# Nota que el resultado sigue siendo un string de bytes
print(f"Encrypted AES ciphertext: {aes_ciphertext}")

Encrypted AES ciphertext: b'`\x8e\xab\x93\xb1\x96n\\\xffkD;w\x8a)\xb0\xe3\xc5\xab2\x0c\x97o\x94yyx\xf2$A\x93\x85\xf2\xf9\xffL\x85<\xa3\x1e\x15\xebLxx\x00\xb7\xaa'


We now show how to decypher the message. Let us try first with the simpler scenario, with a message properly defined to have the adequate bytes length. The logic is similar to the encryption steps.

In [20]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext_) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext_ = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext_}")

Decrypted AES plaintext: no one except my dude can never see this message


Note that if you try to decrypt the padded message, the plaintext will be likely wrong. 

In [21]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext}")

Decrypted AES plaintext: nobody can see this message now or nev


We need to remove the padding!

In [23]:
# Create the Cipher object for decryption
cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = cipher.decryptor()

# Decrypt the ciphertext
decrypted_padded_plaintext = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# Remove padding
unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
unpadded_plaintext = unpadder.update(decrypted_padded_plaintext) + unpadder.finalize()

# Convert bytes back to a string
decrypted_message = unpadded_plaintext.decode('utf-8')

# Print the decrypted message
print(f"Decrypted message: {decrypted_message}")

Decrypted message: nobody can see this message now or never
